# COMS 4995_002 Deep Learning Assignment 1
Due on Thursday, Feb 8, 11:59pm

This assignment can be done in groups of at most 2 students. Everyone must submit on Courseworks individually.

Write down the UNIs of your group (if applicable)

Member 1: Leon Stilwell, ls3223

Member 2: Saahil Jain, sj2675

In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import scipy.misc
import glob
import sys
# you shouldn't need to make any more imports

In [112]:
class NeuralNetwork(object):
    """
    Abstraction of neural network.
    Stores parameters, activations, cached values. 
    Provides necessary functions for training and prediction. 
    """
    def __init__(self, layer_dimensions, drop_prob=0.0, reg_lambda=0.0):
        """
        Initializes the weights and biases for each layer
        :param layer_dimensions: (list) number of nodes in each layer
        :param drop_prob: drop probability for dropout layers. Only required in part 2 of the assignment
        :param reg_lambda: regularization parameter. Only required in part 2 of the assignment
        """
        np.random.seed(1)
        
        self.parameters = {}
        self.num_layers = 1
        self.drop_prob = drop_prob
        self.reg_lambda = reg_lambda
        
        
        # init parameters
        for layer in range(0, self.num_layers + 1):
            curActivationUnits = layer_dimensions[layer]
            print(curActivationUnits)
            if(i!=0):
                prevActivationUnits = layer_dimensions[layer - 1]
            weight = np.random.normal(size=layer_dimensions[layer])
            bias = np.zeros(curActivationUnits)
            self.parameters[layer] = [weight, bias]

    def affineForward(self, A, W, b):
        """
        Forward pass for the affine layer.
        :param A: input matrix, shape (L, S), where L is the number of hidden units in the previous layer and S is
        the number of samples
        :returns: the affine product WA + b, along with the cache required for the backward pass
        """
        print(W.shape)
        print(A.shape)
        Z = np.dot(W, A) + b
        return Z, [W, A, b, Z]
        
    def activationForward(self, A, activation="relu"):
        """
        Common interface to access all activation functions.
        :param A: input to the activation function
        :param prob: activation funciton to apply to A. Just "relu" for this assignment.
        :returns: activation(A)
        """ 
        if(activation == "relu"):
            ret = relu(A)
        return ret

    def relu(self, X):
        A = np.maximum(0, X)
        return A
            
    def dropout(self, A, prob):
        """
        :param A:
        :param prob: drop prob
        :returns: tuple (A, M) 
            WHERE
            A is matrix after applying dropout
            M is dropout mask, used in the backward pass
        """
        M = np.random.rand(A.shape[0], A.shape[1])
        M = (M > prob) * 1.0
        M /= (1 - prob)
        A *= M
        return A, M

    def forwardPropagation(self, X):
        """
        Runs an input X through the neural network to compute activations
        for all layers. Returns the output computed at the last layer along
        with the cache required for backpropagation.
        :returns: (tuple) AL, cache
            WHERE 
            AL is activation of last layer
            cache is cached values for each layer that
                     are needed in further steps
        """
        cache = []
        Z, cacheLayer = self.affineForward(X, self.parameters[0][0], self.parameters[0][1])
        A = activationForward(Z)
        cache.append(cacheLayer)
        for layer in range(1, self.num_layers + 1):
            Z, cacheLayer = self.affineForward(A, self.parameters[layer][0], self.parameters[layer][1])
            A = activationForward(Z)
            cache.append(cacheLayer)
        AL = A
        
        return AL, cache
    
    def costFunction(self, AL, y):
        """
        :param AL: Activation of last layer, shape (num_classes, S)
        :param y: labels, shape (S)
        :param alpha: regularization parameter
        :returns cost, dAL: A scalar denoting cost and the gradient of cost
        """
        # compute loss
        m = y.shape[0]
        AL_softmax = softmax(AL)
        correct_label_prob = AL_softmax[y, range(m)]
        cost = -np.sum(np.log(correct_label_prob)) / m
        
        if self.reg_lambda > 0:
            # add regularization
            reg = "not implemented yet"
        
        # gradient of cost
        dAL = np.multiply(np.subtract(y, AL), -1)
        return cost, dAL
    
    def softmax(X):
        return np.exp(X) / np.sum(np.exp(X), axis = 0)

    def affineBackward(self, dA_prev, cache):
        """
        Backward pass for the affine layer.
        :param dA_prev: gradient from the next layer.
        :param cache: cache returned in affineForward
        :returns dA: gradient on the input to this layer
                 dW: gradient on the weights
                 db: gradient on the bias
        """
        W = cache[0]
        A = cache[1]
        b = cache[2]
        Z = cache[3]
        dZ_prev = np.multiply(dA_prev, relu_derivative(Z))
        dA = np.multiply(W.transpose(), dZ_prev)
        dW = np.multiply(dZ_prev, A.transpose())
        db = dZ_prev
        
        return dA, dW, db

    def activationBackward(self, dA, cache, activation="relu"):
        """
        Interface to call backward on activation functions.
        In this case, it's just relu. 
        """
        if (activation == "relu"):
            relu_derivative()

        
    def relu_derivative(self, dx, cached_x):
        if (dx - cached_x > 0):
            dx = 1
        if(dx - cached_x < 0):
            dx = 0
        return dx

    def dropout_backward(self, dA, cache):
        

        return dA

    def backPropagation(self, dAL, Y, cache):
        """
        Run backpropagation to compute gradients on all paramters in the model
        :param dAL: gradient on the last layer of the network. Returned by the cost function.
        :param Y: labels
        :param cache: cached values during forwardprop
        :returns gradients: dW and db for each weight/bias
        """
        gradients = []
        
        for i in range(num_layers):
            dA, dW, db = affineBackward(cache[num_layers-i])
            gradients.append((dA,dW,db))
            
            #if self.drop_prob > 0:
                #call dropout_backward
           
            
        #if self.reg_lambda > 0:
            # add gradients from L2 regularization to each dW
        
        return gradients


    def updateParameters(self, gradients, alpha):
        """
        :param gradients: gradients for each weight/bias
        :param alpha: step size for gradient descent 
        """
        # W = W - alpha * gradients [for each layer]
        weights = parameters.layer[weight]
        for i in range(len(weights)):
            weights[i] = weights[i] - alpha * gradients[i]
            bias[i] = bias[i] - alpha * gradients[i]
       
    def train(self, X, y, iters=1000, alpha=0.0001, batch_size=100, print_every=100):
        """
        :param X: input samples, each column is a sample
        :param y: labels for input samples, y.shape[0] must equal X.shape[1]
        :param iters: number of training iterations
        :param alpha: step size for gradient descent
        :param batch_size: number of samples in a minibatch
        :param print_every: no. of iterations to print debug info after
        """
        
        for i in range(0, iters):
            # get minibatch
            X_mini, y_mini = self.get_batch(X,y,batch_size)
            print("got batch")
            # forward prop
            AL,cache = self.forwardPropagation(X_mini)
            print("forward prop")
            # compute loss
            cost, dAL = self.lossFunction(AL,y_mini)
            # compute gradients
            gradients = self.backPropagation(dAL,y_mini,cache)
            # update weights and biases based on gradient
            self.updateParameters(gradients,alpha)
            if i % print_every == 0:
                # print cost, train and validation set accuracies
                print(cost)
                print(predict(X))
            
    
    def get_batch(self, X, y, batch_size):
        """
        Return minibatch of samples and labels

        :param X, y: samples and corresponding labels
        :parma batch_size: minibatch size
        :returns: (tuple) X_batch, y_batch
        """
        for i in range(0,X.shape[0],batch_size):
            x_mini = X[i:i+batch_size]
            y_mini = X[i:i+batch_size]

        return x_mini, y_mini
    
    def predict(self, X):
        """
        Make predictions for each sample
        """

        return y_pred

 

In [37]:
# Helper functions, DO NOT modify this

def get_img_array(path):
    """
    Given path of image, returns it's numpy array
    """
    return scipy.misc.imread(path)

def get_files(folder):
    """
    Given path to folder, returns list of files in it
    """
    filenames = [file for file in glob.glob(folder+'*/*')]
    filenames.sort()
    return filenames

def get_label(filepath, label2id):
    """
    Files are assumed to be labeled as: /path/to/file/999_frog.png
    Returns label for a filepath
    """
    tokens = filepath.split('/')
    label = tokens[-1].split('_')[1][:-4]
    if label in label2id:
        return label2id[label]
    else:
        sys.exit("Invalid label: " + label)

In [38]:
# Functions to load data, DO NOT change these

def get_labels(folder, label2id):
    """
    Returns vector of labels extracted from filenames of all files in folder
    :param folder: path to data folder
    :param label2id: mapping of text labels to numeric ids. (Eg: automobile -> 0)
    """
    files = get_files(folder)
    y = []
    for f in files:
        y.append(get_label(f,label2id))
    return np.array(y)

def one_hot(y, num_classes=10):
    """
    Converts each label index in y to vector with one_hot encoding
    """
    y_one_hot = np.zeros((y.shape[0], num_classes))
    y_one_hot[y] = 1
    return y_one_hot.T

def get_label_mapping(label_file):
    """
    Returns mappings of label to index and index to label
    The input file has list of labels, each on a separate line.
    """
    with open(label_file, 'r') as f:
        id2label = f.readlines()
        id2label = [l.strip() for l in id2label]
    label2id = {}
    count = 0
    for label in id2label:
        label2id[label] = count
        count += 1
    return id2label, label2id

def get_images(folder):
    """
    returns numpy array of all samples in folder
    each column is a sample resized to 30x30 and flattened
    """
    files = get_files(folder)
    images = []
    count = 0
    
    for f in files:
        count += 1
        if count % 10000 == 0:
            print("Loaded {}/{}".format(count,len(files)))
        img_arr = get_img_array(f)
        #img_arr = img_arr.flatten() / 255.0
        images.append(img_arr)
    X = np.column_stack(images)

    return X

def get_train_data(data_root_path):
    """
    Return X and y
    """
    train_data_path = data_root_path + 'train'
    id2label, label2id = get_label_mapping(data_root_path+'labels.txt')
    print(label2id)
    X = get_images(train_data_path)
    y = get_labels(train_data_path, label2id)
    return X, y

def save_predictions(filename, y):
    """
    Dumps y into .npy file
    """
    np.save(filename, y)

In [99]:
# Load the data
data_root_path = 'cifar10-hw1//'
X_train, y_train = get_train_data(data_root_path) # this may take a few minutes
X_test = get_images(data_root_path + 'test')
print('Data loading done')

{'horse': 7, 'frog': 6, 'cat': 3, 'bird': 2, 'deer': 4, 'airplane': 0, 'dog': 5, 'truck': 9, 'automobile': 1, 'ship': 8}


c:\users\leo\appdata\local\programs\python\python35-32\lib\site-packages\ipykernel_launcher.py:7: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.
  import sys


Loaded 10000/50000
Loaded 20000/50000
Loaded 30000/50000
Loaded 40000/50000
Loaded 50000/50000
Loaded 10000/10000
Data loading done


## Part 1

#### Simple fully-connected deep neural network

In [113]:
#layer_dimensions = [X_train.shape[0], ..., 10]  # including the input and output layers
layer_dimensions = [X_train.shape[0], 40, 10]
NN = NeuralNetwork(layer_dimensions)
NN.train(X_train, y_train, iters=100, alpha=.001, batch_size=100, print_every=10)

32
40
got batch
(32,)
(32, 1600000, 3)


MemoryError: 

In [ ]:
# y_predicted = NN.predict(X_test)
save_predictions('ans1-uni', y_predicted)

In [ ]:
# test if your numpy file has been saved correctly
loaded_y = np.load('ans1-uni.npy')
print(loaded_y.shape)
loaded_y[:10]

## Part 2: Improving the performance

In [ ]:
NN2 = NeuralNetwork(layer_dimensions, drop_prob=0, reg_lambda=0)
NN2.train(X_train, y_train, iters=1000, alpha=0.00001, batch_size=1000, print_every=10)

In [ ]:
y_predicted2 = NN2.predict(X)
save_predictions(y_predicted, 'ans2-uni')

Write down results for Part 2 here:
...